In [ ]:
//==============================================================
// test测试入口
//==============================================================
#编辑这两个文件，一个是host，一个是device
#test_host.cpp
#test_device.cpp
#生成的可执行是
#testexe

In [ ]:
d

In [1]:
%%writefile Grobner/test_host.cpp
//==============================================================
// Grobner: DPC++ Host
//==============================================================
#include <CL/sycl.hpp>
#include <getopt.h>
#include <ctime>
#include <chrono>
#include <fstream>
#include <sstream>
#include <time.h>
#include <omp.h>
#include<pmmintrin.h>
#include<xmmintrin.h>
#include<immintrin.h>

using namespace std;
using namespace sycl;

#define NUM_THREADS 8
#define COL 3799
#define ROW1 2759
#define ROW2 1953

#define FILE1 "Grobner/6_3799_2759_1953/1.txt"
#define FILE2 "Grobner/6_3799_2759_1953/2.txt"
#define FILE3 "Grobner/6_3799_2759_1953/3.txt"
#define FILEOUT1 "Grobner/output/output1.txt"
#define FILEOUT2 "Grobner/output/output2.txt"
/*
FILE11 "/home/sTest/data/Groebner/1_130_22_8/1.txt"
FILE12 "/home/sTest/data/Groebner/1_130_22_8/2.txt"
FILE13 "/home/sTest/data/Groebner/1_130_22_8/3.txt"
FILE21 "/home/sTest/data/Groebner/2_254_106_53/1.txt"
FILE22 "/home/sTest/data/Groebner/2_254_106_53/2.txt"
FILE23 "/home/sTest/data/Groebner/2_254_106_53/3.txt"
FILE31 "/home/sTest/data/Groebner/3_562_170_53/1.txt"
FILE32 "/home/sTest/data/Groebner/3_562_170_53/2.txt"
FILE33 "/home/sTest/data/Groebner/3_562_170_53/3.txt"
FILE41 "/home/sTest/data/Groebner/4_1011_539_263/1.txt"
FILE42 "/home/sTest/data/Groebner/4_1011_539_263/2.txt"
FILE43 "/home/sTest/data/Groebner/4_1011_539_263/3.txt"
FILE51 "/home/sTest/data/Groebner/5_2362_1226_453/1.txt"
FILE52 "/home/sTest/data/Groebner/5_2362_1226_453/2.txt"
FILE53 "/home/sTest/data/Groebner/5_2362_1226_453/3.txt"
FILE61 "/home/sTest/data/Groebner/6_3799_2759_1953/1.txt"
FILE62 "/home/sTest/data/Groebner/6_3799_2759_1953/2.txt"
FILE63 "/home/sTest/data/Groebner/6_3799_2759_1953/3.txt"
FILE71 "/home/sTest/data/Groebner/7_8399_6375_4535/1.txt"
FILE72 "/home/sTest/data/Groebner/7_8399_6375_4535/2.txt"
FILE73 "/home/sTest/data/Groebner/7_8399_6375_4535/3.txt"
FILE81 "/home/sTest/data/Groebner/8_23075_18748_14325/1.txt"
FILE82 "/home/sTest/data/Groebner/8_23075_18748_14325/2.txt"
FILE83 "/home/sTest/data/Groebner/8_23075_18748_14325/3.txt"
FILE91 "/home/sTest/data/Groebner/9_37960_29304_14921/1.txt"
FILE92 "/home/sTest/data/Groebner/9_37960_29304_14921/2.txt"
FILE93 "/home/sTest/data/Groebner/9_37960_29304_14921/3.txt"
FILEA1 "/home/sTest/data/Groebner/10_43577_39477_54274/1.txt"
FILEA2 "/home/sTest/data/Groebner/10_43577_39477_54274/2.txt"
FILEA3 "/home/sTest/data/Groebner/10_43577_39477_54274/3.txt"
FILEB1 "/home/sTest/data/Groebner/11_85401_5724_756/1.txt"
FILEB2 "/home/sTest/data/Groebner/11_85401_5724_756/2.txt"
FILEB3 "/home/sTest/data/Groebner/11_85401_5724_756/3.txt"
*/
void readFile(bool** M1, bool** M2) {

    ifstream myfile1(FILE1, ios::in);
    if (!myfile1.is_open()) {
        cout << "open file 1 falied" << '\n';
    }
    string str;
    int row = 0;
    int index;
    while (getline(myfile1, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            if (ss >> index) {
                row = index;
                M1[row][index] = true;
            }
            while (ss >> index)
                M1[row][index] = true;
        }
    }
    myfile1.close();

    ifstream myfile2(FILE2, ios::in);
    if (!myfile2.is_open()) {
        cout << "open file 2 falied" << '\n';
    }
    row = 0;
    while (getline(myfile2, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            while (ss >> index)
                M2[row][index] = true;
        }
        row++;
    }
    myfile2.close();
}
void deMap(bool** M, int approach) {
    ofstream myfile;
    switch (approach) {
    case 1:myfile.open(FILEOUT1, ios::in); break;
    case 2:myfile.open(FILEOUT2, ios::in); break;
    default:break;
    }
    if (!myfile.is_open()) {
        cout << "open outputFile falied" << '\n';
    }
    int num = 0;
    for (int i = 0; i < ROW2; i++) {
        for (int j = COL-1; j >= 0; j--) {
            if (M[i][j])
                cout << j << " ";
        }
        cout << "\n";
    }
    myfile.close();
}
void ori(bool** M1, bool** M2) {

    for (int i = 0; i < ROW2; i++) {
        for (int j = COL-1; j >=0; j--) {
            //如果有元素就处理
            if (M2[i][j]) {
                //存在消元子
                if(M1[j][j]){
                    //第i行被消元行M[i][k]和第j行消元子M[j][k]从j开始依次异或
                    for (int k = j; k >= 0; k--) {
                        M2[i][k] = M1[j][k] != M2[i][k];
                    }
                }
                //不存在消元子，升格为消元子,从M[i][j]到M[i][0]结束,拷贝到M[j][k],M[j][0]
                else {
                    for (int k = j; k >= 0; k--) {
                        M1[j][k] = M2[i][k];
                    }
                    //升格完毕，下一行
                    break;
                }
            }
        }
    }
}
void simd(bool** M1, bool** M2) {
    __m128 t1,t2;
    for (int i = 0; i < ROW2; i++) {
        for (int j = COL-1; j >=0; j--) {
            //如果有元素就处理
            if (M2[i][j]) {
                //存在消元子
                if(M1[j][j]){
                    int k;
                    //第i行被消元行M[i][k]和第j行消元子M[j][k]从j开始依次异或
                    for (k = j-16; k >= 0; k-=16) {
                        t1 = _mm_loadu_ps((const float*)&M1[j][k]);
                        t2 = _mm_loadu_ps((const float*)&M2[i][k]);
                        t1 = _mm_xor_ps(t1, t2);
                        _mm_storeu_ps((float*)&M2[i][k],t1);
                    }
                    for(k = k + 15;k>=0;k--){
                        M2[i][k] = M1[j][k] != M2[i][k];
                    }
                }
                //不存在消元子，升格为消元子,从M[i][j]到M[i][0]结束,拷贝到M[j][k],M[j][0]
                else {
                    for (int k = j; k >= 0; k--) {
                        M1[j][k] = M2[i][k];
                    }
                    //升格完毕，下一行
                    break;
                }
            }
        }
    }
}
void omp(bool** M1, bool** M2) {

    for (int i = 0; i < ROW2; i++) {
        for (int j = COL-1; j >=0; j--) {
            //如果有元素就处理
            if (M2[i][j]) {
                //存在消元子
                if(M1[j][j]){
                    //第i行被消元行M[i][k]和第j行消元子M[j][k]从j开始依次异或
                    #pragma omp parallel for shedule(guided,500) num_threads(NUM_THREADS)
                    for (int k = j; k >= 0; k--) {
                        M2[i][k] = M1[j][k] != M2[i][k];
                    }
                }
                //不存在消元子，升格为消元子,从M[i][j]到M[i][0]结束,拷贝到M[j][k],M[j][0]
                else {
                    for (int k = j; k >= 0; k--) {
                        M1[j][k] = M2[i][k];
                    }
                    //升格完毕，下一行
                    break;
                }
            }
        }
    }
}
int main(int argc, char *argv[]) {
    
    //# Define queue with default device for offloading computation
    //queue q(property::queue::enable_profiling{});
    //cpu_selector selector;
    //gpu_selector selector;
    //default_selector selector;
    //host_selector selector;
    
    //cpu_selector selector;
    //queue q(selector);

    //# Print the device name
    //std::cout << "Device: " << q.get_device().get_info<info::device::name>() << "\n";
    queue q;
    std::cout << "Device: " << q.get_device().get_info<info::device::name>() << "\n";
    bool** M1;
    bool** M2;
    M1 = new bool* [COL];
    M2 = new bool* [ROW2];
    for (int i = 0; i < COL; i++) {
        M1[i] = new bool[COL];
        for (int j = 0; j < COL; j++)
            M1[i][j] = false;
    }
    for (int i = 0; i < ROW2; i++) {
        M2[i] = new bool[COL];
        for (int j = 0; j < COL; j++)
            M2[i][j] = false;
    }
    
    //# get start time
    auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    
    readFile(M1,M2);
    auto duration_read = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Read Duration      : " << duration_read / 1e+9 << " seconds\n";
    ori(M1,M2);
    //# Call matrix multiplication kernel implementation
    auto duration_ori = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Compute Duration      : " << duration_ori / 1e+9 << " seconds\n";
    //deMap(M2,1);
    //# print kernel compute duration from host
    return 0;
}

Overwriting Grobner/test_host.cpp


In [ ]:
%%writefile Grobner/test_device.cpp
//==============================================================
// Grobner: DPC++ Device
//==============================================================
#include <CL/sycl.hpp>
using namespace sycl;

void mm_kernel(queue &q, std::vector<float> &matrix_a, std::vector<float> &matrix_b, std::vector<float> &matrix_c, size_t N, size_t M) {
  
}

In [ ]:
! chmod 755 q; chmod 755 run_test.sh; if [ -x "$(command -v qsub)" ]; then ./q run_test.sh "{device.value}"; else ./run_test.sh; fi

In [ ]:
!qstat

In [ ]:
!vim run_test.sh.o1948795

In [ ]:
//==============================================================
// Grobner程序入口
//==============================================================
#编辑这两个文件，一个是host，一个是device
#grobner_dpcpp_host.cpp
#grobner_dpcpp_device.cpp
#生成的可执行是
#grobnerexe

In [3]:
%%writefile Grobner/grobner_dpcpp_host.cpp
//==============================================================
// Grobner: DPC++ Host
//==============================================================


#include <CL/sycl.hpp>
#include <getopt.h>
#include <ctime>
#include <chrono>
#include <fstream>
#include <sstream>
#include <time.h>
#include <omp.h>
#include<pmmintrin.h>
#include<xmmintrin.h>
#include<immintrin.h>

using namespace std;
using namespace sycl;

#define NUM_THREADS 8
#define COL 3799
#define ROW1 2759
#define ROW2 1953

#define FILE1 "Grobner/6_3799_2759_1953/1.txt"
#define FILE2 "Grobner/6_3799_2759_1953/2.txt"
#define FILE3 "Grobner/6_3799_2759_1953/3.txt"
#define FILEOUT1 "Grobner/output/output1.txt"
#define FILEOUT2 "Grobner/output/output2.txt"
/*
FILE11 "/home/sTest/data/Groebner/1_130_22_8/1.txt"
FILE12 "/home/sTest/data/Groebner/1_130_22_8/2.txt"
FILE13 "/home/sTest/data/Groebner/1_130_22_8/3.txt"
FILE21 "/home/sTest/data/Groebner/2_254_106_53/1.txt"
FILE22 "/home/sTest/data/Groebner/2_254_106_53/2.txt"
FILE23 "/home/sTest/data/Groebner/2_254_106_53/3.txt"
FILE31 "/home/sTest/data/Groebner/3_562_170_53/1.txt"
FILE32 "/home/sTest/data/Groebner/3_562_170_53/2.txt"
FILE33 "/home/sTest/data/Groebner/3_562_170_53/3.txt"
FILE41 "/home/sTest/data/Groebner/4_1011_539_263/1.txt"
FILE42 "/home/sTest/data/Groebner/4_1011_539_263/2.txt"
FILE43 "/home/sTest/data/Groebner/4_1011_539_263/3.txt"
FILE51 "/home/sTest/data/Groebner/5_2362_1226_453/1.txt"
FILE52 "/home/sTest/data/Groebner/5_2362_1226_453/2.txt"
FILE53 "/home/sTest/data/Groebner/5_2362_1226_453/3.txt"
FILE61 "/home/sTest/data/Groebner/6_3799_2759_1953/1.txt"
FILE62 "/home/sTest/data/Groebner/6_3799_2759_1953/2.txt"
FILE63 "/home/sTest/data/Groebner/6_3799_2759_1953/3.txt"
FILE71 "/home/sTest/data/Groebner/7_8399_6375_4535/1.txt"
FILE72 "/home/sTest/data/Groebner/7_8399_6375_4535/2.txt"
FILE73 "/home/sTest/data/Groebner/7_8399_6375_4535/3.txt"
FILE81 "/home/sTest/data/Groebner/8_23075_18748_14325/1.txt"
FILE82 "/home/sTest/data/Groebner/8_23075_18748_14325/2.txt"
FILE83 "/home/sTest/data/Groebner/8_23075_18748_14325/3.txt"
FILE91 "/home/sTest/data/Groebner/9_37960_29304_14921/1.txt"
FILE92 "/home/sTest/data/Groebner/9_37960_29304_14921/2.txt"
FILE93 "/home/sTest/data/Groebner/9_37960_29304_14921/3.txt"
FILEA1 "/home/sTest/data/Groebner/10_43577_39477_54274/1.txt"
FILEA2 "/home/sTest/data/Groebner/10_43577_39477_54274/2.txt"
FILEA3 "/home/sTest/data/Groebner/10_43577_39477_54274/3.txt"
FILEB1 "/home/sTest/data/Groebner/11_85401_5724_756/1.txt"
FILEB2 "/home/sTest/data/Groebner/11_85401_5724_756/2.txt"
FILEB3 "/home/sTest/data/Groebner/11_85401_5724_756/3.txt"
*/

//# Define vectors for matricies
std::vector<bool> M1(COL*COL, false);
std::vector<bool> M2(COL*ROW2, false);
    
void readFile() {

    ifstream myfile1(FILE1, ios::in);
    if (!myfile1.is_open()) {
        cout << "open file 1 falied" << '\n';
    }
    string str;
    int row = 0;
    int index;
    while (getline(myfile1, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            if (ss >> index) {
                row = index;
                M1[row*COL+index] = true;
            }
            while (ss >> index)
                M1[row*COL+index] = true;
        }
    }
    myfile1.close();

    ifstream myfile2(FILE2, ios::in);
    if (!myfile2.is_open()) {
        cout << "open file 2 falied" << '\n';
    }
    row = 0;
    while (getline(myfile2, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            while (ss >> index)
                M2[row*COL+index] = true;
        }
        row++;
    }
    myfile2.close();
}
//# matrix multiplication kernel implementation in mm_dpcpp_*.cpp
void gpu_queue() {
    gpu_selector selector;
    queue q(selector);
    device my_device = q.get_device();
    std::cout << "Device: " << my_device.get_info<info::device::name>() << std::endl;
    for (int i = 0; i < ROW2; i++) {
        for (int j = COL-1; j >=0; j--) {
            //如果有元素就处理
            if (M2[i*COL+j]) {
                //存在消元子
                if(M1[j*COL+j]){
                    //第i行被消元行M[i][k]和第j行消元子M[j][k]从j开始依次异或
                    auto R = range<1>(N);
                    {
                        //Create host_ptr buffers for M1 and M2
                        buffer buf_M1(M1,{property::buffer::use_host_ptr()});
                        buffer buf_M2(M2,{property::buffer::use_host_ptr()});    
    
                        q.submit([&](handler& h) {
                            //create Accessors for a and b
                            accessor d_M1(buf_M1,h,read_only);
                            accessor d_M2(buf_M2,h);        
                            h.parallel_for(range{(unsigned long)(j+1)}, [=](id<1> idx){ 
                                int k = idx[0] + j; // 等同于for(int k = j ; k>=0; k--)
                                d_M2[i*COL+k] = d_M1[j*COL+k] != d_M2[i*COL+k];
                                }); 
                        });
                    }
                }
                //不存在消元子，升格为消元子,从M[i][j]到M[i][0]结束,拷贝到M[j][k],M[j][0]
                else {
                    for (int k = j; k >= 0; k--) {
                        M1[j*COL+k] = M2[i*COL+k];
                    }
                    //升格完毕，下一行
                    break;
                }
            }
        }
    }
}
int main(int argc, char *argv[]) {
    
    
    
    //# get start time
    auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    
    readFile();
    auto duration_read = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Read Duration      : " << duration_read / 1e+9 << " seconds\n";
    gpu_queue();
    //# Call matrix multiplication kernel implementation
    auto duration_ori = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Compute Duration      : " << duration_ori / 1e+9 << " seconds\n";
    deMap(M2,1);
    //# print kernel compute duration from host
}


Overwriting Grobner/grobner_dpcpp_host.cpp


In [ ]:
%%writefile Grobner/grobner_dpcpp_host.cpp
//==============================================================
// Grobner: DPC++ Host
//==============================================================


#include <CL/sycl.hpp>
#include <getopt.h>
#include <ctime>
#include <chrono>
#include <fstream>
#include <sstream>
#include <time.h>
#include <omp.h>
#include<pmmintrin.h>
#include<xmmintrin.h>
#include<immintrin.h>

using namespace std;
using namespace sycl;

#define NUM_THREADS 8
#define COL 3799
#define ROW1 2759
#define ROW2 1953

#define FILE1 "Grobner/6_3799_2759_1953/1.txt"
#define FILE2 "Grobner/6_3799_2759_1953/2.txt"
#define FILE3 "Grobner/6_3799_2759_1953/3.txt"
#define FILEOUT1 "Grobner/output/output1.txt"
#define FILEOUT2 "Grobner/output/output2.txt"
/*
FILE11 "/home/sTest/data/Groebner/1_130_22_8/1.txt"
FILE12 "/home/sTest/data/Groebner/1_130_22_8/2.txt"
FILE13 "/home/sTest/data/Groebner/1_130_22_8/3.txt"
FILE21 "/home/sTest/data/Groebner/2_254_106_53/1.txt"
FILE22 "/home/sTest/data/Groebner/2_254_106_53/2.txt"
FILE23 "/home/sTest/data/Groebner/2_254_106_53/3.txt"
FILE31 "/home/sTest/data/Groebner/3_562_170_53/1.txt"
FILE32 "/home/sTest/data/Groebner/3_562_170_53/2.txt"
FILE33 "/home/sTest/data/Groebner/3_562_170_53/3.txt"
FILE41 "/home/sTest/data/Groebner/4_1011_539_263/1.txt"
FILE42 "/home/sTest/data/Groebner/4_1011_539_263/2.txt"
FILE43 "/home/sTest/data/Groebner/4_1011_539_263/3.txt"
FILE51 "/home/sTest/data/Groebner/5_2362_1226_453/1.txt"
FILE52 "/home/sTest/data/Groebner/5_2362_1226_453/2.txt"
FILE53 "/home/sTest/data/Groebner/5_2362_1226_453/3.txt"
FILE61 "/home/sTest/data/Groebner/6_3799_2759_1953/1.txt"
FILE62 "/home/sTest/data/Groebner/6_3799_2759_1953/2.txt"
FILE63 "/home/sTest/data/Groebner/6_3799_2759_1953/3.txt"
FILE71 "/home/sTest/data/Groebner/7_8399_6375_4535/1.txt"
FILE72 "/home/sTest/data/Groebner/7_8399_6375_4535/2.txt"
FILE73 "/home/sTest/data/Groebner/7_8399_6375_4535/3.txt"
FILE81 "/home/sTest/data/Groebner/8_23075_18748_14325/1.txt"
FILE82 "/home/sTest/data/Groebner/8_23075_18748_14325/2.txt"
FILE83 "/home/sTest/data/Groebner/8_23075_18748_14325/3.txt"
FILE91 "/home/sTest/data/Groebner/9_37960_29304_14921/1.txt"
FILE92 "/home/sTest/data/Groebner/9_37960_29304_14921/2.txt"
FILE93 "/home/sTest/data/Groebner/9_37960_29304_14921/3.txt"
FILEA1 "/home/sTest/data/Groebner/10_43577_39477_54274/1.txt"
FILEA2 "/home/sTest/data/Groebner/10_43577_39477_54274/2.txt"
FILEA3 "/home/sTest/data/Groebner/10_43577_39477_54274/3.txt"
FILEB1 "/home/sTest/data/Groebner/11_85401_5724_756/1.txt"
FILEB2 "/home/sTest/data/Groebner/11_85401_5724_756/2.txt"
FILEB3 "/home/sTest/data/Groebner/11_85401_5724_756/3.txt"
*/

//# Define vectors for matricies
std::vector<bool> M1(COL*COL, false);
std::vector<bool> M2(COL*ROW2, false);
    
void readFile() {

    ifstream myfile1(FILE1, ios::in);
    if (!myfile1.is_open()) {
        cout << "open file 1 falied" << '\n';
    }
    string str;
    int row = 0;
    int index;
    while (getline(myfile1, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            if (ss >> index) {
                row = index;
                M1[row*COL+index] = true;
            }
            while (ss >> index)
                M1[row*COL+index] = true;
        }
    }
    myfile1.close();

    ifstream myfile2(FILE2, ios::in);
    if (!myfile2.is_open()) {
        cout << "open file 2 falied" << '\n';
    }
    row = 0;
    while (getline(myfile2, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            while (ss >> index)
                M2[row*COL+index] = true;
        }
        row++;
    }
    myfile2.close();
}
//# matrix multiplication kernel implementation in mm_dpcpp_*.cpp
void gpu_queue() {
    gpu_selector selector;
    queue q(selector);
    device my_device = q.get_device();
    std::cout << "Device: " << my_device.get_info<info::device::name>() << std::endl;
    for (int i = 0; i < ROW2; i++) {
        for (int j = COL-1; j >=0; j--) {
            //如果有元素就处理
            if (M2[i*COL+j]) {
                //存在消元子
                if(M1[j*COL+j]){
                    //第i行被消元行M[i][k]和第j行消元子M[j][k]从j开始依次异或
                    auto R = range<1>(N);
                    {
                        //Create host_ptr buffers for M1 and M2
                        buffer buf_M1(M1,{property::buffer::use_host_ptr()});
                        buffer buf_M2(M2,{property::buffer::use_host_ptr()});    
    
                        q.submit([&](handler& h) {
                            //create Accessors for a and b
                            accessor d_M1(buf_M1,h,read_only);
                            accessor d_M2(buf_M2,h);        
                            h.parallel_for(range{(unsigned long)(j+1)}, [=](id<1> idx){ 
                                int k = idx[0] + j; // 等同于for(int k = j ; k>=0; k--)
                                d_M2[i*COL+k] = d_M1[j*COL+k] != d_M2[i*COL+k];
                                }); 
                        });
                    }
                }
                //不存在消元子，升格为消元子,从M[i][j]到M[i][0]结束,拷贝到M[j][k],M[j][0]
                else {
                    for (int k = j; k >= 0; k--) {
                        M1[j*COL+k] = M2[i*COL+k];
                    }
                    //升格完毕，下一行
                    break;
                }
            }
        }
    }
}
int main(int argc, char *argv[]) {
    
    //# get start time
    auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    readFile();
    auto duration_read = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Read Duration      : " << duration_read / 1e+9 << " seconds\n";
    gpu_queue();
    //# Call matrix multiplication kernel implementation
    auto duration_ori = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Compute Duration      : " << duration_ori / 1e+9 << " seconds\n";
    deMap(M2,1);
}


In [ ]:
! chmod 755 q; chmod 755 run_grobner.sh; if [ -x "$(command -v qsub)" ]; then ./q run_grobner.sh "{device.value}"; else ./run_grobner.sh; fi

In [ ]:
%%writefile Grobner/grobner_dpcpp_host.cpp
//==============================================================
// Grobner: DPC++ Host
//==============================================================


#include <CL/sycl.hpp>
#include <getopt.h>
#include <ctime>
#include <chrono>
#include <fstream>
#include <sstream>
#include <time.h>
#include <omp.h>
#include<pmmintrin.h>
#include<xmmintrin.h>
#include<immintrin.h>

using namespace std;
using namespace sycl;

#define NUM_THREADS 8
#define COL 3799
#define ROW1 2759
#define ROW2 1953

#define FILE1 "Grobner/6_3799_2759_1953/1.txt"
#define FILE2 "Grobner/6_3799_2759_1953/2.txt"
#define FILE3 "Grobner/6_3799_2759_1953/3.txt"
#define FILEOUT1 "Grobner/output/output1.txt"
#define FILEOUT2 "Grobner/output/output2.txt"
/*
FILE11 "/home/sTest/data/Groebner/1_130_22_8/1.txt"
FILE12 "/home/sTest/data/Groebner/1_130_22_8/2.txt"
FILE13 "/home/sTest/data/Groebner/1_130_22_8/3.txt"
FILE21 "/home/sTest/data/Groebner/2_254_106_53/1.txt"
FILE22 "/home/sTest/data/Groebner/2_254_106_53/2.txt"
FILE23 "/home/sTest/data/Groebner/2_254_106_53/3.txt"
FILE31 "/home/sTest/data/Groebner/3_562_170_53/1.txt"
FILE32 "/home/sTest/data/Groebner/3_562_170_53/2.txt"
FILE33 "/home/sTest/data/Groebner/3_562_170_53/3.txt"
FILE41 "/home/sTest/data/Groebner/4_1011_539_263/1.txt"
FILE42 "/home/sTest/data/Groebner/4_1011_539_263/2.txt"
FILE43 "/home/sTest/data/Groebner/4_1011_539_263/3.txt"
FILE51 "/home/sTest/data/Groebner/5_2362_1226_453/1.txt"
FILE52 "/home/sTest/data/Groebner/5_2362_1226_453/2.txt"
FILE53 "/home/sTest/data/Groebner/5_2362_1226_453/3.txt"
FILE61 "/home/sTest/data/Groebner/6_3799_2759_1953/1.txt"
FILE62 "/home/sTest/data/Groebner/6_3799_2759_1953/2.txt"
FILE63 "/home/sTest/data/Groebner/6_3799_2759_1953/3.txt"
FILE71 "/home/sTest/data/Groebner/7_8399_6375_4535/1.txt"
FILE72 "/home/sTest/data/Groebner/7_8399_6375_4535/2.txt"
FILE73 "/home/sTest/data/Groebner/7_8399_6375_4535/3.txt"
FILE81 "/home/sTest/data/Groebner/8_23075_18748_14325/1.txt"
FILE82 "/home/sTest/data/Groebner/8_23075_18748_14325/2.txt"
FILE83 "/home/sTest/data/Groebner/8_23075_18748_14325/3.txt"
FILE91 "/home/sTest/data/Groebner/9_37960_29304_14921/1.txt"
FILE92 "/home/sTest/data/Groebner/9_37960_29304_14921/2.txt"
FILE93 "/home/sTest/data/Groebner/9_37960_29304_14921/3.txt"
FILEA1 "/home/sTest/data/Groebner/10_43577_39477_54274/1.txt"
FILEA2 "/home/sTest/data/Groebner/10_43577_39477_54274/2.txt"
FILEA3 "/home/sTest/data/Groebner/10_43577_39477_54274/3.txt"
FILEB1 "/home/sTest/data/Groebner/11_85401_5724_756/1.txt"
FILEB2 "/home/sTest/data/Groebner/11_85401_5724_756/2.txt"
FILEB3 "/home/sTest/data/Groebner/11_85401_5724_756/3.txt"
*/

//# Define vectors for matricies
std::vector<bool> M1(COL*COL, false);
std::vector<bool> M2(COL*ROW2, false);
    
void readFile(bool* M1,bool* M2) {

    ifstream myfile1(FILE1, ios::in);
    if (!myfile1.is_open()) {
        cout << "open file 1 falied" << '\n';
    }
    string str;
    int row = 0;
    int index;
    while (getline(myfile1, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            if (ss >> index) {
                row = index;
                M1[row*COL+index] = true;
            }
            while (ss >> index)
                M1[row*COL+index] = true;
        }
    }
    myfile1.close();

    ifstream myfile2(FILE2, ios::in);
    if (!myfile2.is_open()) {
        cout << "open file 2 falied" << '\n';
    }
    row = 0;
    while (getline(myfile2, str)) {
        stringstream ss;
        ss << str;
        if (!ss.eof()) {
            while (ss >> index)
                M2[row*COL+index] = true;
        }
        row++;
    }
    myfile2.close();
}
//# matrix multiplication kernel implementation in mm_dpcpp_*.cpp
void usm(bool* M1,bool* M2,queue q) {
    for (int i = 0; i < ROW2; i++) {
        for (int j = COL-1; j >=0; j--) {
            //如果有元素就处理
            if (M2[i*COL+j]) {
                //存在消元子
                if(M1[j*COL+j]){
                    //第i行被消元行M[i][k]和第j行消元子M[j][k]从j开始依次异或
                     q.parallel_for(range{(unsigned long)(j+1)}, [=](id<1> idx){ 
                         int k = idx[0] + j; // 等同于for(int k = j ; k>=0; k--)
                         M2[i*COL+k] = M1[j*COL+k] != M2[i*COL+k];
                     }); 
                }
                //不存在消元子，升格为消元子,从M[i][j]到M[i][0]结束,拷贝到M[j][k],M[j][0]
                else {
                    for (int k = j; k >= 0; k--) {
                        M1[j*COL+k] = M2[i*COL+k];
                    }
                    //升格完毕，下一行
                    break;
                }
            }
        }
    }
}
int main(int argc, char *argv[]) {
    gpu_selector selector;
    queue q(selector);
    bool* M1 = (bool *)malloc_shared(COL*COL* sizeof(bool), q);
    bool* M2 = (bool *)malloc_shared(COL*ROW2* sizeof(bool), q);
    //# get start time
    auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    readFile(M1,M2);
    auto duration_read = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Read Duration      : " << duration_read / 1e+9 << " seconds\n";
    usm(M1,M2,q);
    //# Call matrix multiplication kernel implementation
    auto duration_ori = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Compute Duration      : " << duration_ori / 1e+9 << " seconds\n";
    deMap(M2,1);
}


In [ ]:
%%writefile Grobner/grobner_dpcpp_host.cpp
//==============================================================
// Grobner: DPC++ Host
//==============================================================


#include <CL/sycl.hpp>
#include <getopt.h>
#include <ctime>
#include <chrono>

using namespace sycl;

//# matrix multiplication kernel implementation in mm_dpcpp_*.cpp
void mm_kernel(queue &q, std::vector<float> &matrix_a, std::vector<float> &matrix_b, std::vector<float> &matrix_c, size_t N, size_t M);

int main(int argc, char *argv[]) {
    
    size_t N = 1024;
    size_t M = 16;
    int VERIFY = 0;
    int PRINT_OUTPUT_MATRIX = 0;
    
    int arg;
    while ((arg = getopt (argc, argv, "n:m:vp")) != -1)
        switch (arg){
            case 'n':
                N = std::atoi(optarg);
                break;
            case 'm':
                M = std::atoi(optarg);
                break;
            case 'v':
                VERIFY = 1;
                break;
            case 'p':
                PRINT_OUTPUT_MATRIX = 1;
                break;
            case 'h':
                std::cout << std::endl;
                std::cout << "Usage   : ./a.out -n <MATRIX_SIZE> -m <WORK_GROUP_SIZE> -v -p\n\n";
                std::cout << "          [-n] size for matrix, eg: 1024\n";
                std::cout << "          [-m] size of work_group, eg: 8/16\n";
                std::cout << "          [-v] verify output with linear computation on cpu\n";
                std::cout << "          [-p] print output matrix\n";
                std::cout << "Example : ./a.out -n 1024 -m 16 -v -p\n\n";
                std::exit(0);
        }

    //# Define vectors for matricies
    std::vector<float> matrix_a(N*N);
    std::vector<float> matrix_b(N*N);
    std::vector<float> matrix_c(N*N);
    std::vector<float> matrix_d(N*N);
    
    //# Initialize matricies with values
    float v1 = 2.f;
    float v2 = 3.f;
    for (int i=0; i<N; i++)
        for (int j=0; j<N; j++){
            matrix_a[i*N+j] = v1++;
            matrix_b[i*N+j] = v2++;
            matrix_c[i*N+j] = 0.f;
            matrix_d[i*N+j] = 0.f;
    }
    
    //# Define queue with default device for offloading computation
    queue q(property::queue::enable_profiling{});
    std::cout << "Offload Device        : " << q.get_device().get_info<info::device::name>() << "\n";
    std::cout << "max_work_group_size   : " << q.get_device().get_info<info::device::max_work_group_size>() << "\n";
    
    //# get start time
    auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    
    //# Call matrix multiplication kernel implementation
    mm_kernel(q, matrix_a, matrix_b, matrix_c, N, M);
    
    //# print kernel compute duration from host
    auto duration = std::chrono::high_resolution_clock::now().time_since_epoch().count() - start;
    std::cout << "Compute Duration      : " << duration / 1e+9 << " seconds\n";
    
    //# Print Output if -p in cmd-line
    if (PRINT_OUTPUT_MATRIX){
        for (int i=0; i<N; i++){
            for (int j=0; j<N; j++){
                std::cout << matrix_c[i*N+j] << " ";
            }
            std::cout << "\n";
        }
    } else {
        std::cout << " [0][0] = " << matrix_c[0] << "\n";
    }
    
    //# Compute local and compare with offload computation if -v in cmd-line
    if (VERIFY){
        int fail = 0;
        for(int i=0; i<N; i++){
            for (int j = 0; j < N; j++) {
                for(int k=0; k<N; k++){
                    matrix_d[i*N+j] += matrix_a[i*N+k] * matrix_b[k*N+j];
                }
                if(matrix_c[i*N+j] != matrix_d[i*N+j]) fail = 1;
            }
        }
        if(fail == 1){
            std::cout << "FAIL\n";
        } else {
            std::cout << "PASS\n";
        }
    }
    return 0;
}


In [ ]:
%%writefile Grobner/grobner_dpcpp_device.cpp
//==============================================================
// Grobner: DPC++ Device
//==============================================================
#include <CL/sycl.hpp>
using namespace sycl;

void mm_kernel(queue &q, std::vector<float> &matrix_a, std::vector<float> &matrix_b, std::vector<float> &matrix_c, size_t N, size_t M) {
    std::cout << "Configuration         : MATRIX_SIZE= " << N << "x" << N << " | WORK_GROUP_SIZE= " << M << "x" << M << "\n";

    //# Create buffers for matricies
    buffer a(matrix_a);
    buffer b(matrix_b);
    buffer c(matrix_c);

    //# Submit command groups to execute on device
    auto e = q.submit([&](handler &h){
        //# Create accessors to copy buffers to the device
        auto A = a.get_access<access::mode::read>(h);
        auto B = b.get_access<access::mode::read>(h);
        auto C = c.get_access<access::mode::write>(h);

        //# Define size for ND-range and work-group size
        range<2> global_size(N,N);
        range<2> work_group_size(M,M);

        //# Create local accessors
        accessor<float, 2, access::mode::read_write, access::target::local> A_tile(range<2>(M, M), h);
        accessor<float, 2, access::mode::read_write, access::target::local> B_tile(range<2>(M, M), h);

        //# Parallel Compute Matrix Multiplication
        h.parallel_for(nd_range<2>{global_size, work_group_size}, [=](nd_item<2> item){
            const int i = item.get_global_id(0);
            const int j = item.get_global_id(1);
            const int x = item.get_local_id(0);
            const int y = item.get_local_id(1);

            float temp = 0.f;
            int k;
            for (int t = 0; t < N; t+=M) {
                A_tile[x][y] = A[i * N + (t + y)];
                B_tile[x][y] = B[(t + x) * N + j];
                item.barrier(access::fence_space::local_space);
                for (k = 0; k < M; k++) {
                    temp += A_tile[x][k] * B_tile[k][y];
                }
                item.barrier(access::fence_space::local_space);
            }
            C[i*N+j] = temp;
        });
    });
    c.get_access<access::mode::read>();
    
    //# print kernel compute duration from event profiling
    auto kernel_duration = (e.get_profiling_info<info::event_profiling::command_end>() - e.get_profiling_info<info::event_profiling::command_start>());
    std::cout << "Kernel Execution Time : " << kernel_duration / 1e+9 << " seconds\n";
}
